返回dfoff每个userid中，【只领券，核销数，直接买，单商铺最大购买次数，成交商家数】的数量

office 2 2368.7907860279083s  
office 2 2276.1418340206146 s  
home  1  2320  
home  2  1503.2748665809631 s  
home  2  1475.3602182865143 s  
home  6  1271.9127905368805 s  
home  8  1239.525927066803 s  
home 12  1383.0938215255737 s  

In [2]:
import sys 
sys.path.append('../app/')
import fileio
import preprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time
import threading

In [3]:
# 读取数据
dfoff ,dftest,dfon = fileio.read_all('../data/')

# 特征处理
dfoff = preprocess.get_label(dfoff)
dfoff = preprocess.process_manjian(dfoff)

user_off_list = dfoff.User_id.unique()

In [4]:
def user_features(uid):
    """返回userid，只领券，核销数，直接买，单商铺最大购买次数，成交商家数"""
    lcdict = {0:0,1:0,-1:0}
    df001 = dfoff[dfoff.User_id==uid]
    
    dflabelcount = df001.label.value_counts()
    dfMerchantcount = df001.Merchant_id.value_counts()
    for i in dflabelcount.index:
        lcdict[i] = dflabelcount[i]
    
    return uid,lcdict[0],lcdict[1],lcdict[-1],dfMerchantcount.max(),dfMerchantcount.count()

class myThread(threading.Thread):
    def __init__(self,threadID,name,user_off_list):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.user_off_list = user_off_list

        
    def run(self):
        print("开始线程"+self.name)
        self.obj = test_fun(self.user_off_list,self.name)
        print("退出线程"+self.name)
        
def test_fun(user_off_list,name):
    n = 0.
    ll = []
    for uid in  user_off_list:
        if n%1000==0:
            print("{0}---{1}%".format(name,n*100/len(user_off_list)) )
        ll.append(user_features(uid))
        n = n+1
    return ll

In [5]:

# 创建新线程
halfnum = int(len(user_off_list)/2)

#def earthworm(core_num=2):
#    user
thread1 = myThread(1, "Thread-1",user_off_list[0:halfnum])
thread2 = myThread(2, "Thread-2",user_off_list[halfnum:])


time_start=time.time()
# 开启新线程
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print ("退出主线程")

time_end=time.time()
print('time cost',time_end-time_start,'s')

开始线程Thread-1
Thread-1---0.0%
开始线程Thread-2
Thread-2---0.0%
Thread-1---0.3707562314853607%
Thread-2---0.3707562314853607%
Thread-1---0.7415124629707214%
Thread-2---0.7415124629707214%
Thread-1---1.112268694456082%
Thread-2---1.112268694456082%
Thread-1---1.4830249259414428%
Thread-2---1.4830249259414428%
Thread-1---1.8537811574268035%
Thread-2---1.8537811574268035%
Thread-1---2.224537388912164%
Thread-2---2.224537388912164%
Thread-1---2.5952936203975248%
Thread-2---2.5952936203975248%
Thread-1---2.9660498518828855%
Thread-2---2.9660498518828855%
Thread-1---3.3368060833682462%
Thread-2---3.3368060833682462%
Thread-1---3.707562314853607%
Thread-2---3.707562314853607%
Thread-1---4.078318546338967%
Thread-2---4.078318546338967%
Thread-1---4.449074777824328%
Thread-2---4.449074777824328%
Thread-1---4.819831009309689%
Thread-2---4.819831009309689%
Thread-1---5.1905872407950495%
Thread-2---5.1905872407950495%
Thread-1---5.56134347228041%
Thread-2---5.56134347228041%
Thread-1---5.932099703765771

Thread-1---50.052091250523695%
Thread-2---50.052091250523695%
Thread-1---50.422847482009054%
Thread-2---50.422847482009054%
Thread-1---50.79360371349441%
Thread-2---50.79360371349441%
Thread-1---51.16435994497977%
Thread-2---51.16435994497977%
Thread-1---51.53511617646514%
Thread-2---51.53511617646514%
Thread-1---51.9058724079505%
Thread-2---51.9058724079505%
Thread-1---52.276628639435856%
Thread-2---52.276628639435856%
Thread-1---52.647384870921215%
Thread-2---52.647384870921215%
Thread-1---53.01814110240658%
Thread-2---53.01814110240658%
Thread-1---53.38889733389194%
Thread-2---53.38889733389194%
Thread-1---53.7596535653773%
Thread-2---53.7596535653773%
Thread-1---54.13040979686266%
Thread-2---54.13040979686266%
Thread-1---54.501166028348024%
Thread-2---54.501166028348024%
Thread-1---54.87192225983338%
Thread-2---54.87192225983338%
Thread-1---55.24267849131874%
Thread-2---55.24267849131874%
Thread-1---55.6134347228041%
Thread-2---55.6134347228041%
Thread-1---55.98419095428947%
Thread

In [6]:
dflist = [pd.DataFrame(dfs) for dfs in [thread1.obj,thread2.obj]]
dfobj = pd.concat(dflist)
dfobj.index = range(0,len(dfobj))
dfobj.columns = ['用户id','只领券','核销数','直接买','单商铺最大购买次数','成交商家数']

In [7]:
dfobj.head()

,用户id,只领券,核销数,直接买,单商铺最大购买次数,成交商家数
0,1439408,5,0,2,6,2
1,1832624,1,0,0,1,1
2,2029232,3,0,2,3,3
3,2747744,1,0,0,1,1
4,196342,1,0,1,2,1


In [11]:
path = '../storage/newfeatures/dfoff_add_f1/'
fileio.ifmakdir(path)
dfobj.to_csv(path+"new_fea_1.csv",index = 0)